### Read data and visualize
Process
 1. Read data from wikipedia
 2. Create nx.graph
 3. Visuzlize

three options for reading data,
- option 0:directory from python dict
- option 1:from rdb
- option 2:from elasticsearch


### 1. Read data from wikipedia

In [ ]:
import pandas as pd
from wikiscraper import WikiScraper
data = pd.read_csv("contents_test.csv",index_col=0)
wscs = []

j = 0
start = 0
stop = start+1+len(data.index.values)

for i in data.index.values:
    j+=1
    if j<start:
        continue
        
    wsc = WikiScraper()
    wsc.load_wiki(i.decode('utf-8'), pageid=data.at[i, 'id'], lang='ja')
#         wsc.load_wiki(i.decode('utf-8'), 'ja')
# #     wsc.load_html(p.html())
    if wsc._bsObj is not None:
        wsc.get_list_from_headline(u'.*スタッフ')
        wsc.get_table('infobox')
    
    wscs.append(wsc)
    
    print '-------------------------------------'
    print wsc
    print '+++++++++++++++++++++'
#     print wsc._name

    if j>=stop:
        break

### 2.Create nx.graph

three options for reading data,
- option 0:directory from python dict
- option 1:from rdb
- option 2:from elasticsearch

In [ ]:
#option 0 directory create nx.graph from python dict

threshold = 1.0
max_number_of_nodes = 500
count_once = True
import re
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()

key_rm = [u'放送期間', u'放送時間', u'公開', u'上映時間', u'次作', u'回数', u'放送分']
value_rm = [u'', u'同上', u'日本', u'日本語', u'英語', u'公式サイト', u'ほか', u'ステレオ放送', u'文字多重放送',u'歴代エンディングテーマを参照',
            u'フジテレビ番組基本情報']

for wsc in wscs:
    
    point = float(data.at[wsc._name.encode('utf-8'), 'ポイント'])
    G.add_node(wsc._name, genre='content', point=point)
    attrs = set()
    for k, v in wsc._result.items():
        #add first genre to node attribute
        if k == u'ジャンル':
            if len(v)==0:
                G.node[wsc._name]['genre'] = v
            else:
                G.node[wsc._name]['genre'] = v[0]
        elif k not in key_rm:
            for attr in v:
                if attr not in data.index.values and attr not in value_rm: #do nothing for contents itself
                    if G.node.get(attr) is None: #first time
                        G.add_node(attr, genre='attribute', point=point)
                    elif not (attr in attrs and count_once): # from second time
                        G.node[attr]['point'] += point
                    G.add_edge(wsc._name, attr, relation=k) 
                    attrs.add(attr)
    print wsc._name, len(G.nodes)
        
cliques = nx.find_cliques(G)
# #標準出力
# for c in cliques:
#     print c



In [ ]:
#option 1 use database
# 1-1 put data into db

#Write python dict to database
#use psycopg not pandas nor sqlalchemy since, column is dynamic
#temporary ignore captial or not
import traceback
from dbcontrol import DBControl

table_name = 'contents'
dbc = DBControl(dbname='testdb', host='localhost', user='pgadmin', password='password')
dbc.create_table(table_name)
i = 0
# print get_table_name()
for wsc in wscs:
    print i,
    i+=1
    name = value_pre_process(wsc._name) 
    dbc.insert(table_name, 'name', name)
    for k, v in wsc._result.items():
#         print '--------------------------------------'
#         print k, ','.join(set(v))
        k = key_pre_process(k)
        value = value_pre_process(','.join(set(v)))
        if (k,) not in dbc.get_columns_name(table_name):
            try:
                dbc.alter(table_name, k, 'text')
            except:
                print 'error in alter key:', name,  k
                traceback.print_exc()
                dbc._cur.execute('ABORT;')
                continue
        try:
            dbc.update(table_name, k, value, name)
        except:
            print 'error in update value:', name, k, value 
            traceback.print_exc()
            dbc._cur.execute('ABORT;')
            continue
        

In [ ]:
#option 1 use database
# 1-2 create graph from db

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from dbcontrol import DBControl, key_pre_process, value_pre_process

G = nx.Graph()
count_once=True
key_rm = [u'放送期間', u'放送時間', u'公開', u'上映時間', u'次作', u'回数', u'放送分']
value_rm = [u'', u'同上', u'日本', u'日本語', u'英語', u'公式サイト', u'ほか', u'ステレオ放送', u'文字多重放送',u'歴代エンディングテーマを参照',
            u'フジテレビ番組基本情報']

#1. read name and point from csv
input_data = pd.read_csv("contents_test.csv",index_col=0)
names = input_data.index.values

#2. get data from database
table_name = 'contents'
dbc = DBControl(dbname='testdb', host='localhost', user='pgadmin', password='password')

#3. separate key and value
#4. calc
columns = dbc.get_columns_name(table_name)
# name = 'ウンナンさん'
for name in names:
    point = float(input_data.at[name, 'ポイント'])
    G.add_node(name, genre='content', point=point)
    attrs = set()

    print '--------',name, point, '----------'
    for column in columns:
        if column[0]=='name':
            continue
        res = dbc.get_value(table_name, key='name', value=value_pre_process(name.decode('utf-8')), column=column[0])
        k = column[0].decode('utf-8')
        if len(res)>0:
            val = res[0][0]
            if val is not None:
#                 print 'key:', k,[k]
                val = val.decode('utf-8')
                v = val.split(u'，')
                if k == u'ジャンル':
                    if len(v)==0:
                        G.node[name]['genre'] = v
                    else:
                        G.node[name]['genre'] = v[0]
                elif k not in key_rm:
                    for attr in v:
#                         print '   value:', attr, [attr]
                        if attr not in names and attr not in value_rm: #do nothing for contents itself
                            if G.node.get(attr) is None: #first time
                                G.add_node(attr, genre='attribute', point=point)
                            elif not (attr in attrs and count_once): # from second time
                                G.node[attr]['point'] += point
                            G.add_edge(name, attr, relation=k) 
                            attrs.add(attr)

#                 print col, '::', 
#                 for v in val.split('，'):
#                     print v, ' = ',
#                 print
        else:
            pass #todo not in the database

        
#to confirm
# for e in G.edgfrom dbcontrol import DBControl
# else:
#     print e[0], e[1]
# for n in G.nodes:
#     print n

In [ ]:
#option2 use elasticsearch
# 2-1 put data into elasticsearch
from elasticsearch import Elasticsearch
es = Elasticsearch()

table_name = 'contents'
# mapping = {
#   "mappings": {
#     "contents": {
#       "properties": {
#         "name": {
#           "type":     "keyword",
# #           "analyzer": "kuromoji",
#           "index" : "not_analyzed"
#         }
#       }
#     }
#   }
# }
i = 0
# print get_table_name()
for wsc in wscs:
    print i,
    i+=1
    res_dict = wsc._result
    print [wsc._name]
    res_dict['name'] = wsc._name 
    es.index(index="my-index", body=res_dict, doc_type="_doc")
#     print res_dict

In [ ]:
#read from elasticsearch
# 2-2 create nx.graph from elasticsearch
from elasticsearch import Elasticsearch
es = Elasticsearch()

res = es.search(index="my-index", body=search_query)
for hit in res['hits']['hits']:
    print '-------------'
    print hit['_source']['name']
    for k, v in hit['_source'].items():
        if k != 'name':
            print k,
            for value in v:
                print value, ', ',
            print



In [57]:
#read from elasticsearch
# 2-2 create nx.graph from elasticsearch

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()
count_once=True
key_rm = [u'放送期間', u'放送時間', u'公開', u'上映時間', u'次作', u'回数', u'放送分']
value_rm = [u'', u'同上', u'日本', u'日本語', u'英語', u'公式サイト', u'ほか', u'ステレオ放送', u'文字多重放送',u'歴代エンディングテーマを参照',
            u'フジテレビ番組基本情報']

#1. read name and point from csv
input_data = pd.read_csv("contents_test.csv",index_col=0)
names = input_data.index.values

#2. get data from elastic search
from elasticsearch import Elasticsearch
es = Elasticsearch()

# res = es.search(index="my-index", body={"query": {"match_all": {}}})
# res = es.search(index="my-index", body={"query": {"match": {"name":u'ウンナンさん'}}})
# res = es.search(index="my-index", body={"query": {"term": {'name': u"ウンナンさん"} }})
search_query = {
    "query":{
        "query_string":{
            "default_field" : "name",
            "query":"\"ウンナンさん\""
        }
    }
}

for name in names:
    
    point = float(input_data.at[name, 'ポイント'])
    G.add_node(name, genre='content', point=point)
    attrs = set()
    search_query['query']['query_string']['query'] = '\"' + name + '\"'
    res = es.search(index="my-index", body=search_query)
    for hit in res['hits']['hits']:
        print '-------------'
        print hit['_source']['name']
        for k, v in hit['_source'].items():
#             if k != 'name':
#                 print k,
#                 for value in v:
#                     print value, ', ',
#                 print
            #add first genre to node attribute
            if k == u'ジャンル':
                if len(v)==0:
                    G.node[name]['genre'] = v
                else:
                    G.node[name]['genre'] = v[0]
            elif k not in key_rm and  k != 'name':
                for attr in v:
                    if attr not in names and attr not in value_rm: #do nothing for contents itself
                        if G.node.get(attr) is None: #first time
                            G.add_node(attr, genre='attribute', point=point)
                        elif not (attr in attrs and count_once): # from second time
                            G.node[attr]['point'] += point
                        
                        G.add_edge(name, attr, relation=k) 
                        attrs.add(attr)
        


-------------
ウンナンさん
-------------
ウンナンの気分は上々。
-------------
UN街
-------------
勇者ヨシヒコと魔王の城
-------------
勇者ヨシヒコと悪霊の鍵
-------------
勇者ヨシヒコと導かれし七人
-------------
ネリさまぁ〜ず
-------------
神さまぁ〜ず
-------------
ホリさまぁ〜ず
-------------
マルさまぁ〜ず
-------------
バナナ塾
-------------
バナナマンのブログ刑事
-------------
オトナ養成所_バナナスクール
-------------
ツギクルもん
-------------
うつけもん
-------------
オサレもん
-------------
30minutes
-------------
30minutes鬼
-------------
30minutes鬼
-------------
デリパンダ〜おしゃべりデリ坊、東京ド真ん中配達中〜
-------------
あらびき団
-------------
タイプライターズ
-------------
飛び出せ!科学くん
-------------
ラブレターズのオールナイトニッポン0(ZERO)


/home/kevinh/.local/lib/python2.7/site-packages/ipykernel_launcher.py:58: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


### 3. Visuzlize

1. remove nodes 
2. visualize

In [58]:
# remove nodes for visualization
threshold = 1.0
max_number_of_nodes = 500
count_once = True
first = True
while len(G.nodes)>max_number_of_nodes or first:
    print 'Threshold: ', threshold
    print ' node num before simplify:', len(G.nodes)
    value_rm.extend([node for node,degree in dict(G.degree()).items() if degree < 2 and G.node[node]['genre'] is 'attribute']) #remove node based on degree
    value_rm.extend([node for node in G.nodes if  G.node[node]['point'] < threshold and G.node[node]['genre'] is 'attribute'])
    G.remove_nodes_from(value_rm)
    print ' node num after simplify:', len(G.nodes)
    threshold += 0.1
    first = False

#mix the closed genres
for v in data.index.values:
    title = v.decode('utf-8')
    if title in G.node:
        if re.search( u'(.*バラエティ.*)|(.*お笑い.*)', G.node[title]['genre']):
            G.node[title]['genre'] = u'バラエティ'
        if re.search( u'.*ドラマ.*', G.node[title]['genre']):
            G.node[title]['genre'] = u'ドラマ'
        if re.search( u'.*SF.*', G.node[title]['genre']):
            G.node[title]['genre'] = u'SF'

# nx.draw_networkx(G,font_family='AppleGothic',font_size =8)
# plt.show()
# nx.write_gexf(G, 'result.gexf')

Threshold:  1.0
 node num before simplify: 645
 node num after simplify: 42


In [59]:
#visualize with pyvis + networkx 
from pyvis.network import Network
# nxg = nx.complete_graph(10)
# gg = Network(notebook=True)
# gg.from_nx(G)
# gg.show("nx.html")

def get_net_node(net, key, value):
    for item in net.nodes:
        if item[key] == value:
            return item
    return None 

net = Network(height='500px', width='1000px')
# net = Network(notebook=True)
net.show_buttons(filter_=['physics'])
net.force_atlas_2based(
    gravity=-30, 
    central_gravity=0.01, 
    spring_length=1.0, 
    spring_strength=0.8, 
    damping=1.0, 
    overlap=1.0)
# net.options.physics.__dict__['timestep'] = 0.5
net.options.physics.__dict__['adaptiveTimestep'] = True
net.toggle_hide_edges_on_drag(True)

scaling = dict(min=1, 
                       max=100, 
                       label=dict(enable=True, 
                                          min=10, 
                                          max=100))

node_id = 0
for label in G.nodes:
#     print label, G.node[label]['genre'], G.node[label]['point']
    if G.node[label]['genre'] == 'attribute':
        shape = 'dot'
        value = G.node[label]['point']
        mass=G.node[label]['point']
    else:
        shape = 'box'
        value = threshold
        mass = 10
    net.add_node(node_id, label=label, 
                 group=G.node[label]['genre'], 
                 value=value,
                 mass=mass,
                 scaling = scaling,
                 shape=shape)
    node_id += 1
    
for edge in G.edges:
    edge[0], edge[1]
    net.add_edge( get_net_node(net, 'label', edge[0])['id'], 
                 get_net_node(net, 'label', edge[1])['id'])
    
# net.save_graph("result.html")
net.show("result.html")

/home/kevinh/.local/lib/python2.7/site-packages/ipykernel_launcher.py:10: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  # Remove the CWD from sys.path while we load stuff.
